In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuration
DATA_PATH = "generated_images_1_1/"  # Directory with 0/ and 1/ subdirectories
MODEL_PATH ="best_model_des.pth"  # Path to your saved model
IMG_SIZE = (256, 256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
class DenseCNN(nn.Module):
    def __init__(self, base_channels=32):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, base_channels, 3, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels, base_channels*2, 3, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*2, base_channels*4, 3, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*4, base_channels*8, 3, padding=1),
            nn.BatchNorm2d(base_channels*8),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(base_channels*8, 2)
        )
        
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = DenseCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def evaluate_model(directory_path):
    """Evaluate model on images in directory structure"""
    true_labels = []
    pred_labels = []
    confidences = []
    image_paths = []
    
    # Walk through directory structure
    for label in ['0', '1']:
        class_dir = os.path.join(directory_path, label)
        if not os.path.exists(class_dir):
            continue
            
        for img_file in os.listdir(class_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_file)
                
                # Load and preprocess image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                    
                img = cv2.resize(img, IMG_SIZE)
                img_tensor = transform(Image.fromarray(img)).unsqueeze(0).to(device)
                
                # Get prediction
                with torch.no_grad():
                    output = model(img_tensor)
                    probabilities = torch.softmax(output, dim=1)
                    _, predicted = torch.max(output, 1)
                    confidence = probabilities[0][predicted].item()
                
                # Store results
                true_labels.append(int(label))
                pred_labels.append(predicted.item())
                confidences.append(confidence)
                image_paths.append(img_path)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"\nEvaluation Results for {directory_path}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, target_names=['Benign (0)', 'Malignant (1)']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, pred_labels))
    
    # Print some examples
    print("\nSample Predictions:")
    for i in range(min(5, len(image_paths))):
        print(f"Image: {os.path.basename(image_paths[i])} | "
              f"True: {true_labels[i]} | "
              f"Pred: {pred_labels[i]} | "
              f"Confidence: {confidences[i]:.2f}")
    
    return {
        'accuracy': accuracy,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'confidences': confidences,
        'image_paths': image_paths
    }

if __name__ == "__main__":
    print(f"Evaluating model on images in: {DATA_PATH}")
    print(f"Using device: {device}")
    results = evaluate_model(DATA_PATH)

Evaluating model on images in: generated_images_1_1/
Using device: cuda

Evaluation Results for generated_images_1_1/:
Accuracy: 0.8200

Classification Report:
               precision    recall  f1-score   support

   Benign (0)       0.79      0.88      0.83        50
Malignant (1)       0.86      0.76      0.81        50

     accuracy                           0.82       100
    macro avg       0.82      0.82      0.82       100
 weighted avg       0.82      0.82      0.82       100


Confusion Matrix:
[[44  6]
 [12 38]]

Sample Predictions:
Image: generated1_0.png | True: 0 | Pred: 0 | Confidence: 0.94
Image: generated1_1.png | True: 0 | Pred: 0 | Confidence: 0.85
Image: generated1_10.png | True: 0 | Pred: 1 | Confidence: 0.61
Image: generated1_11.png | True: 0 | Pred: 0 | Confidence: 0.97
Image: generated1_12.png | True: 0 | Pred: 0 | Confidence: 0.97


In [2]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuration
DATA_PATH = "generated_images_1_1_1/"  # Directory with 0/ and 1/ subdirectories
MODEL_PATH ="best_model_des.pth"  # Path to your saved model
IMG_SIZE = (256, 256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
class DenseCNN(nn.Module):
    def __init__(self, base_channels=32):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, base_channels, 3, padding=1),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels, base_channels*2, 3, padding=1),
            nn.BatchNorm2d(base_channels*2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*2, base_channels*4, 3, padding=1),
            nn.BatchNorm2d(base_channels*4),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(base_channels*4, base_channels*8, 3, padding=1),
            nn.BatchNorm2d(base_channels*8),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(base_channels*8, 2)
        )
        
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = DenseCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def evaluate_model(directory_path):
    """Evaluate model on images in directory structure"""
    true_labels = []
    pred_labels = []
    confidences = []
    image_paths = []
    
    # Walk through directory structure
    for label in ['0', '1']:
        class_dir = os.path.join(directory_path, label)
        if not os.path.exists(class_dir):
            continue
            
        for img_file in os.listdir(class_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_file)
                
                # Load and preprocess image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                    
                img = cv2.resize(img, IMG_SIZE)
                img_tensor = transform(Image.fromarray(img)).unsqueeze(0).to(device)
                
                # Get prediction
                with torch.no_grad():
                    output = model(img_tensor)
                    probabilities = torch.softmax(output, dim=1)
                    _, predicted = torch.max(output, 1)
                    confidence = probabilities[0][predicted].item()
                
                # Store results
                true_labels.append(int(label))
                pred_labels.append(predicted.item())
                confidences.append(confidence)
                image_paths.append(img_path)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"\nEvaluation Results for {directory_path}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, target_names=['Benign (0)', 'Malignant (1)']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, pred_labels))
    
    # Print some examples
    print("\nSample Predictions:")
    for i in range(min(5, len(image_paths))):
        print(f"Image: {os.path.basename(image_paths[i])} | "
              f"True: {true_labels[i]} | "
              f"Pred: {pred_labels[i]} | "
              f"Confidence: {confidences[i]:.2f}")
    
    return {
        'accuracy': accuracy,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'confidences': confidences,
        'image_paths': image_paths
    }

if __name__ == "__main__":
    print(f"Evaluating model on images in: {DATA_PATH}")
    print(f"Using device: {device}")
    results = evaluate_model(DATA_PATH)

Evaluating model on images in: generated_images_1_1_1/
Using device: cuda

Evaluation Results for generated_images_1_1_1/:
Accuracy: 0.8249

Classification Report:
               precision    recall  f1-score   support

   Benign (0)       0.81      0.83      0.82       143
Malignant (1)       0.84      0.82      0.83       154

     accuracy                           0.82       297
    macro avg       0.82      0.82      0.82       297
 weighted avg       0.83      0.82      0.82       297


Confusion Matrix:
[[118  25]
 [ 27 127]]

Sample Predictions:
Image: 0.png | True: 0 | Pred: 1 | Confidence: 0.62
Image: 1.png | True: 0 | Pred: 1 | Confidence: 0.58
Image: 100.png | True: 0 | Pred: 0 | Confidence: 0.99
Image: 101.png | True: 0 | Pred: 0 | Confidence: 0.97
Image: 102.png | True: 0 | Pred: 0 | Confidence: 0.97
